# Fixture Para Todos

## Valores parametrizables por el usuario

### Cantidad de equipos a realizarles intercambios

In [1]:
n_team_to_be_swapped = 1

### Cantidad de intercambios a realizar por equipo

In [2]:
n_swaps_per_team = 2

### Cantidad de iteraciones del algoritmo

In [3]:
n_iterations = 10

### Cantidad de iteraciones que se prohiben los intercambios previamente realizados

In [4]:
n_forbidden_iterations = 2

## Parámetros de las restricciones

### Restricción 1

Todos los equipos deben jugar la misma cantidad de partidos de local y de visitante que el resto o, a lo sumo, <b>param_r1</b> partido/s de diferencia con respecto al resto de los equipos

In [5]:
param_r1 = 1

### Restricción 2

Todos los equipos deben jugar la misma cantidad de partidos de local y de visitante o, a lo sumo, <b>param_r2</b> partido/s de diferencia entre ambas condiciones

In [6]:
param_r2 = 1

### Restricción 3

Ningún equipo puede jugar más de <b>param_r3</b> partidos consecutivos en la misma condición

In [7]:
param_r3 = 2

### Restricción 4

Los equipos considerados clásicos entre sí no pueden jugar de local en la misma fecha

In [8]:
restrictions_params = {"r1": param_r1, "r2": param_r2, "r3": param_r3}

## Importación de librerías

In [9]:
import csv
import math
from fixture_manager import FixtureManager
from distance_travelled_manager import DistanceTravelledManager

## Carga de archivos CSV

In [10]:
# Carga los equipos a partir del archivo csv
def load_teams(path):
    '''
        Formato de los equipos:
        ["Equipo A", ..., "Equipo Z"]
    '''

    teams = []
    with open(path) as f:
        content = f.readlines()
    teams = [x.strip() for x in content]

    return teams

In [11]:
# Carga las distancias a partir del archivo csv
def load_distances(path):
    '''
        Formato de las distancias:
        {"Equipo A": {"Equipo B": 100, ..., "Equipo Z": 200},
        ...,
        "Equipo X": {"Equipo Y": 150, ..., "Equipo Z": 300}
        }
    '''

    distances = {}
    header = True

    with open(path) as csv_file:
        csv_distances = csv.reader(csv_file, delimiter=",")
        for row in csv_distances:
            if header:
                header = False
                continue

            team1, team2, distance = row[0], row[1], row[2]

            if team1 not in distances:
                distances[team1] = {}
            distances[team1][team2] = distance

            if team2 not in distances:
                distances[team2] = {}
            distances[team2][team1] = distance

    return distances

In [12]:
# Carga los derbies a partir del archivo csv
def load_derbies(path):
    '''
    Formato de los derbies:
        {"Equipo A": "Equipo B",
        ...,
        "Equipo Y": "Equipo Z"
        }
    '''

    derbies = {}
    header = True

    with open(path) as csv_file:
        csv_derbies = csv.reader(csv_file, delimiter=",")
        for row in csv_derbies:
            if header:
                header = False
                continue
            
            team1 = row[0]
            team2 = row[1]
            derbies[team1] = team2
            derbies[team2] = team1

    return derbies

In [13]:
teams = load_teams("csv/test/teams.csv")
distances = load_distances("csv/test/distances.csv")
derbies = load_derbies("csv/test/derbies.csv")

## Creación de clases

In [14]:
fixture_manager = FixtureManager(teams, derbies, restrictions_params, distances)
distance_travelled_manager = DistanceTravelledManager(distances)

## Fixture Inicial

### Generación

In [15]:
initial_fixture = fixture_manager.calculate_initial_fixture()

Fixture incompatible. Reiniciando...


### Validación

In [16]:
fixture_manager.is_compatible(initial_fixture)

True

### Cálculo de la varianza

In [17]:
distance_travelled_manager.calculate_km_traveled_variance(initial_fixture)

997.3635244984649

## Fixtures Iterativos

## Fixture Final

In [18]:
fixture_manager.print(initial_fixture)

--------------------------------------------------------------
--------------------- FIXTURE PARA TODOS ---------------------
--------------------------------------------------------------

Fecha 1:
	River Plate		Vs.		Boca Juniors
	Talleres		Vs.		Belgrano

Fecha 2:
	Belgrano		Vs.		Boca Juniors
	River Plate		Vs.		Talleres

Fecha 3:
	Belgrano		Vs.		River Plate
	Boca Juniors		Vs.		Talleres


### Validación

In [19]:
fixture_manager.is_compatible(initial_fixture)

True

### Varianza total

In [20]:
distance_travelled_manager.calculate_km_traveled_variance(initial_fixture)

997.3635244984649